In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPool2D, AvgPool2D, Dense, Conv2D, Normalization, BatchNormalization, Flatten, Dropout, Input, Activation, Concatenate, Multiply, Add, Input
from tensorflow.keras.models import Model

input = Input(shape= (512,512,3))
x = Conv2D(512, (3,3), padding="same", dilation_rate=(1,1), activation='relu')(input)
x = BatchNormalization()(x)

x1 = Conv2D(256, (3,3), padding="same", activation='relu')(x)
x1 = BatchNormalization()(x1)

x2 = Conv2D(128, (3,3), padding="same", activation='relu')(x1)
xc1 = Concatenate()([input,x1,x2])
xa1 = Activation('sigmoid')(xc1)

xi1 = Conv2D(387, (1,1), padding="same", activation='relu')(input)
xad = Add()([xa1,xi1])

x3 = Conv2D(256, (3,3), padding="same", activation='relu')(xad)

x3 = MaxPool2D()(x3)
x4 = Conv2D(128, (3,3), padding="same", activation='relu')(x3)
x4 = BatchNormalization()(x4)
x5 = Conv2D(64, (3,3), padding="same", activation='relu')(x4)
x5 = BatchNormalization()(x5)
xc2 = Concatenate()([x4,x5])
x0 = Conv2D(128, (3,3), padding="same", activation='relu')(xc2)
x5 = BatchNormalization()(x0)
xa2 = Activation('sigmoid')(x5)

xi2 = Conv2D(128, (1,1), strides=(2,2), padding="same", activation='relu')(input)
xad2 = Add()([xa2,xi2])

x6 = Conv2D(128, (3,3), strides=(2,2), padding="same", activation='relu')(xad2)
x6 = BatchNormalization()(x6)
x6 = Conv2D(64, (3,3), padding="same", activation='relu')(x6)
x6 = BatchNormalization()(x6)
x6 = MaxPool2D()(x6)
x6 = BatchNormalization()(x6)

xi3 = Conv2D(64, (1,1), strides=(8,8), padding="same", dilation_rate=(1,1), activation='relu')(input)
xad3 = Add()([x6,xi3])

x7 = Conv2D(32, (3,3), strides=(2,2), padding="same", activation='relu')(xad3)
x7 = AvgPool2D(2,2)(x7)
x7 = Conv2D(16, (3,3), padding="same", activation='sigmoid')(x7)
out = Conv2D(8, (3,3),  padding="same", activation='softmax')(x7)
model = Model(input, out)
model.summary()
model.save('model.h5')

# Feature Map Extractor

In [ ]:
# model = tf.keras.models.load_model('model.h5')
def featureMapExtractor(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)
    feature_maps = model.predict(img_array)
    feature_map = feature_maps[0, :, :, 0]
    min_value = np.min(feature_map)
    max_value = np.max(feature_map)
    normalized_feature_map1 = (feature_map - min_value) / (max_value - min_value)
    return np.where(normalized_feature_map1 > 0.6, 1, 0)

# Encryption AES256 GCM

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def encrypt_image(image_path_cmp):
    key = get_random_bytes(32)
    with open(image_path_cmp, 'rb') as file:
        original_data = file.read()

    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(original_data)

    file_out = image_path_cmp + '.enc'
    with open(file_out, 'wb') as file:
        [ file.write(x) for x in (cipher.nonce, tag, ciphertext) ]
    return file_out, key

def decrypt_image(key, encrypted_image_path):
    with open(encrypted_image_path, 'rb') as file:
        nonce, tag, ciphertext = [ file.read(x) for x in (16, 16, -1) ]

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    data = cipher.decrypt_and_verify(ciphertext, tag)
    
    file_out = encrypted_image_path[:-4]  
    with open(file_out, 'wb') as file:
        file.write(data)
    return file_out



# IPFS

In [ ]:
import requests
import os
from dotenv import load_dotenv
import pprint

PINATA_JWT_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySW5mb3JtYXRpb24iOnsiaWQiOiJhNWRiODU0MS1kOTMxLTQzNmMtYjhkYy00M2Q3NWJjN2JiZTEiLCJlbWFpbCI6InllYXNpcjQwMkBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwicGluX3BvbGljeSI6eyJyZWdpb25zIjpbeyJpZCI6IkZSQTEiLCJkZXNpcmVkUmVwbGljYXRpb25Db3VudCI6MX0seyJpZCI6Ik5ZQzEiLCJkZXNpcmVkUmVwbGljYXRpb25Db3VudCI6MX1dLCJ2ZXJzaW9uIjoxfSwibWZhX2VuYWJsZWQiOmZhbHNlLCJzdGF0dXMiOiJBQ1RJVkUifSwiYXV0aGVudGljYXRpb25UeXBlIjoic2NvcGVkS2V5Iiwic2NvcGVkS2V5S2V5IjoiNTZjNDZlOTI1MDMxZTI4MDFmNzEiLCJzY29wZWRLZXlTZWNyZXQiOiJkMmQ4ODFiNTM0N2FkOThkMmUwNTJjYjIyNmVjN2FmMmQxN2Q1ZDU4N2MzM2MyODI5OTBhZWZhMzI4NTQwZDFmIiwiaWF0IjoxNzE0MDI4MDYwfQ.1QlnRLCkvAHq7IIshtNFORl0l-D3sYGD_8l786PQqYE"


In [ ]:
def upload_to_pinata_ipfs(encrypted_img):
    url = "https://api.pinata.cloud/pinning/pinFileToIPFS"
    headers = {'Authorization': f'Bearer {PINATA_JWT_TOKEN}'}

    with open(encrypted_img, 'rb') as file:
        response = requests.post(url,files={'file':file},headers=headers)
        return "https://ipfs.io/ipfs/"+response.json()['IpfsHash']

    return response.json()
    

In [ ]:
import os, requests
def download_from_ipfs(ipfs_url):
    response = requests.get(ipfs_url)
    if response.status_code != 200:
        raise Exception("Failed to download file from IPFS")
    
    output_dir = '/kaggle/working'
    encrypted_img_path = os.path.join(output_dir, 'encrypted_image.jpeg.enc')
    with open(encrypted_img_path, 'wb') as file:
        file.write(response.content)
    
    return encrypted_img_path

In [ ]:
# download_from_ipfs('https://ipfs.io/ipfs/QmTKg7RaAqrpw1pidBjSh88yeNzU9ekLg4MueCef2zMY7d')

# Flask

In [ ]:
!pip install pyngrok

In [ ]:
!pip install flask_cors

In [ ]:
from pyngrok import ngrok
ngrok_key = "2h6aGsbaomJpD9RNn1hfjhVHycV_2vDU8c4vhSWrdws92oJik"
port = 5000
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

In [ ]:
from flask import Flask, jsonify, request, render_template_string 
import numpy as np
from PIL import Image
import binascii
import base64
from io import BytesIO
from flask_cors import CORS  
import time

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/integrity_verification', methods=['POST'])
def integrity_verification():
    start_time = time.time()
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    file.save(file.filename)
    image_path = '/kaggle/working/'+file.filename
    fm = featureMapExtractor(image_path)
    data = request.get_json()
    feature_map = data['feature_map']
    integrity = None
    if fm == feature_map:
        integrity = True
    else:
        integrity = False
    end_time = time.time()
    print("Total time for integrity verification:", end_time - start_time)
    return jsonify({'integrity_status': integrity})

@app.route('/get_fm_en_ipfs', methods=['POST'])
def get_fm_en_ipfs():
    start_time = time.time()
    
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    file.save(file.filename)
    cmp_start_time = time.time()
    img = Image.open("/kaggle/working/"+file.filename)
    img.save('compressed_'+file.filename, quality=100, optimize=True, format='JPEG')
    image_path_cmp = '/kaggle/working/compressed_'+file.filename
    cmp_end_time = time.time()
    print("compresstion time:", cmp_end_time - cmp_start_time)
    
    fm_start_time = time.time()
    fm = featureMapExtractor(image_path_cmp)
    fm_end_time = time.time()
    print("featureMapExtractor time:", fm_end_time - fm_start_time)
    
    encrypt_start_time = time.time()
    encrypted_img_path, encryption_key = encrypt_image(image_path_cmp)
    encrypt_end_time = time.time()
    print("encrypt_image time:", encrypt_end_time - encrypt_start_time)
    
    upload_start_time = time.time()
    ipfs_hash = upload_to_pinata_ipfs(encrypted_img_path)
    upload_end_time = time.time()
    print("upload_to_pinata_ipfs time:", upload_end_time - upload_start_time)
    
    encryption_key_hex = encryption_key.hex()
    original_encryption_key = binascii.unhexlify(encryption_key_hex)
    print(fm.tolist())
    end_time = time.time()
    print("Total time for get_fm_en_ipfs:", end_time - start_time)
    
    return jsonify({'feature_map': fm.tolist(), 'encryption_key': encryption_key_hex, 'ipfs_hash': ipfs_hash})

@app.route('/get_decrypted_image', methods=['POST'])
def get_decrypted_image():
    start_time = time.time()
    
    data = request.get_json()
    if not data or 'ipfs_url' not in data or 'encryption_key' not in data:
        return jsonify({'error': 'IPFS URL and encryption key are required'}), 400
    ipfs_url = data['ipfs_url']
    encryption_key = data['encryption_key']
    
    encryption_key = binascii.unhexlify(encryption_key)
    
    download_start_time = time.time()
    encrypted_image_path = download_from_ipfs(ipfs_url)
    download_end_time = time.time()
    print("download_from_ipfs time:", download_end_time - download_start_time)
    
    decrypt_start_time = time.time()
    decrypted_img_path = decrypt_image(encryption_key, encrypted_image_path)
    decrypt_end_time = time.time()
    print("decrypt_image time:", decrypt_end_time - decrypt_start_time)
    
    try:
        with open(decrypted_img_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        end_time = time.time()
        print("Total time for get_decrypted_image:", end_time - start_time)
        
        return jsonify({'decrypted_image': encoded_string})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
if __name__ == '__main__':
    app.run(port=port)